<a href="https://colab.research.google.com/github/mounikaduddukuri/S15/blob/master/Depth_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd 
from keras.optimizers import SGD
from keras.layers import Conv2D, Input, MaxPooling2D as Pool, BatchNormalization as BN, UpSampling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
import keras.backend as K
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
from skimage.io import imread 
from skimage.segmentation import mark_boundaries 
from sklearn.metrics import roc_curve, auc # roc curve tools
from skimage.color import label2rgb
import numpy as np # linear algebra / matrices
# make the notebook interactive
from ipywidgets import interact, interactive, fixed 
import ipywidgets as widgets #add new widgets
from IPython.display import display
import inspect, json
import h5py
import skimage.transform
import scipy
import matplotlib as mpl
import os
import warnings
os.environ['KERAS_BACKEND'] = 'theano'
%matplotlib inline


In [0]:
from keras import backend as K
#K.set_image_dim_ordering('th')
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers import merge, Input
from keras.layers.core import ActivityRegularization
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
MAKE_RGB_LAYER = False

In [0]:
def build_nd_fake_unet(in_shape, layers, depth,
                    conv_op, pool_op, upscale_op,
                   layer_size_fcn = lambda i: 3,
                   pool_size = 2,
                   dropout_rate = 0.0):
    inputs = Input(in_shape)
    
    first_layer = conv_op(depth,3)(conv_op(depth,3)(inputs))
    
    last_layer = first_layer
    
    conv_layers = []
    pool_layers = []
    
    for ilay in range(layers):
        # double filters
        pool_layers += [pool_op(pool_size)(last_layer)]
        lay_depth = depth*np.power(2,ilay+1)
        lay_kern_wid = layer_size_fcn(ilay)
        post_conv_step = conv_op(lay_depth, lay_kern_wid)(conv_op(lay_depth, lay_kern_wid)(pool_layers[-1]))
        if dropout_rate > 0: post_conv_step = Dropout(dropout_rate)(post_conv_step)
        
        conv_layers += [post_conv_step]
        
        last_layer = conv_layers[-1]

    # remove the last layer
    rev_layers = list(reversed(list(zip(range(layers),conv_layers[:-1]))))
    rev_layers += [(-1,first_layer)]
    
    for ilay, l_pool in rev_layers:
        lay_depth = depth*np.power(2,ilay+1)
        cur_up = upscale_op(pool_size)(last_layer)
        cur_merge = merge([cur_up, l_pool], mode='concat', concat_axis=1)
        cur_conv = conv_op(lay_depth, 3)(conv_op(lay_depth, 3)(cur_merge))
        last_layer = cur_conv
    
    
    out_conv = conv_op(1, 1, activation='tanh')(last_layer)
    ar_out = ActivityRegularization(l1=1e-4, l2=1e-2)(out_conv)
    model = Model(input=inputs, output=ar_out)
    
    return model

def build_2d_umodel(in_img, layers, depth = 8, lsf = lambda i: 3, pool_size = 3, dropout_rate = 0):
    conv_op = lambda n_filters, f_width, activation='relu', **kwargs: Convolution2D(n_filters, (f_width, f_width), activation=activation, border_mode='same', **kwargs)
    pool_op = lambda p_size: MaxPooling2D(pool_size=(p_size, p_size))
    upscale_op = lambda p_size: UpSampling2D(size=(p_size, p_size))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        m =build_nd_fake_unet(in_img.shape[1:], layers, depth, conv_op, pool_op, upscale_op,layer_size_fcn = lsf, pool_size = pool_size, dropout_rate = dropout_rate)
    return m

In [0]:
img_size = 150
bg = []
fg = []
dep =[]
testing_data = []
bgfg = []
datadir = '/content/drive/My Drive/dep_est/bg'

#datadir = '/content/drive/My Drive/dep_est/fg'
def train_data(adrs,lis):
	path = adrs
	for img in os.listdir(path):
		img_arr = cv2.imread(os.path.join(path,img))
		new_arr = cv2.resize(img_arr,(img_size,img_size))
		lis.append([new_arr])

train_data(datadir,bg)
#train_data(datadir1,fg)
len(bg),np.shape(bg)

(100, (100, 1, 150, 150, 3))

In [0]:
add = '/content/drive/My Drive/dep_est/nwbgfg'
train_data(add,bgfg)
len(bgfg),np.shape(bgfg[0])

(100, (1, 150, 150, 3))

In [0]:
np.shape(bg)

(100, 1, 150, 150, 3)

In [0]:
train = np.array(bg).reshape(-1,img_size,img_size,3)
print(len(train))
#test = np.array(testing_data).reshape(-1,img_size,img_size,3)

100


In [0]:
y = np.array(bgfg).reshape(-1,img_size,img_size,3)
print(len(y))

100


In [0]:
rgb_images = train
depth_maps = y
train_rgb_images = ((rgb_images.astype(np.float32)-127)/127)
print(train_rgb_images.shape,depth_maps.shape)

(100, 150, 150, 3) (100, 150, 150, 3)


In [0]:
rgbd_model = build_2d_umodel(rgb_images, layers = 5, depth = 4, pool_size = 2)


TypeError: ignored

In [0]:

# overview of network
list(enumerate(map(lambda x: (x.name,('in:',(x.input_shape[1:]),'out:',x.output_shape[1:])),rgbd_model.layers)))

TypeError: ignored

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_111 (Conv2D)          (None, 146, 146, 64)      4864      
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 142, 142, 64)      102464    
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 71, 71, 64)        0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 69, 69, 128)       73856     
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 67, 67, 128)       147584    
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 33, 33, 128)       0         
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 31, 31, 256)      

In [0]:
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer,ZeroPadding2D,Deconvolution2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras import optimizers

model = Sequential()
model.add(Convolution2D(64, 5, 5, activation='relu', input_shape=(150, 150, 3)))
model.add(Convolution2D(64, 5, 5, activation='relu'))
model.add(MaxPooling2D())

#model.add(Dropout(0.5))

model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D())

#model.add(Dropout(0.5))

model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(Convolution2D(256, 3, 3, activation='relu'))

#model.add(Dropout(0.5))

model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(Convolution2D(512, 3, 3, activation='relu'))

#model.add(Dropout(0.5))

model.add(ZeroPadding2D())
model.add(Deconvolution2D(512, 3, 3, (None, 512, 41, 61), subsample=(2, 2), activation='relu'))
model.add(Deconvolution2D(512, 3, 3, (None, 512, 123, 183), subsample=(3, 3), activation='relu'))

#model.add(cropping.Cropping2D(cropping=((1, 2), (11, 12))))

model.add(Convolution2D(1, 1, 1, activation='sigmoid', border_mode='same'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", input_shape=(150, 150,...)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Kera

In [0]:
model.compile(optimizer='adam',loss='mse')

In [0]:

model.fit(rgb_images,depth_maps)

ValueError: ignored

In [0]:
model.fit_generator(rgb_images,depth_maps)

Epoch 1/1


ValueError: ignored